# W266 Final Project Code
# Amazon Product Review Aspect-Based Sentiment
## Jennifer Mahle and Joanna Wang (Sections 3 and 1, respectively) 


In [1]:
import os
import pandas as pd

dataset = "Electronics_5.json"

if os.path.isfile(dataset):
    df = pd.read_json("Electronics_5.json", lines=True)
else:
    url = r"http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics_5.json.gz"
    df = pd.read_json(url, compression='gzip', lines=True)

display(df.tail(10))
df.shape
print(df.info())

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
6739580,5,NaN,True,"07 25, 2017",A1OOVLE2KZ6KGA,B01HJCN1EI,NaN,Puddzee,These are my favorite charging cords for a few...,Worth the price.,1500940800,NaN
6739581,1,NaN,True,"04 4, 2017",A77K1B31UAQ29,B01HJCN1EI,NaN,addictedtoreading,"Update....after 2 months of gentle use, cable ...",UPDATE...BREAKS AND SLOW CHARGING,1491264000,NaN
6739582,3,NaN,True,"07 8, 2017",A2SVXUVUAWUDK2,B01HJH42KU,NaN,Andrew,These are okay. The connection becomes very if...,Hope this makes sense. You'd understand if you...,1499472000,NaN
6739583,2,NaN,True,"05 21, 2017",A12E1JGKV0ETAB,B01HJH42KU,NaN,John Adams,I liked the length and the product at first bu...,Lost ability to connect.,1495324800,NaN
6739584,3,NaN,True,"06 26, 2017",A1HKXEX8BEQC2E,B01HJH40WU,NaN,Dasha stephens,not holding up over time :(,not holding up over time :(,1498435200,NaN
6739585,4,NaN,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,NaN,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,NaN
6739586,4,NaN,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,NaN,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,NaN
6739587,5,2,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,NaN
6739588,5,2,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,NaN,Faith,Works well.,Five Stars,1480377600,NaN
6739589,5,NaN,True,"03 31, 2017",A1FGCIRPRNZWD5,B01HJF704M,NaN,Brando,I have it plugged into a usb extension on my g...,Works well enough..,1490918400,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6739590 entries, 0 to 6739589
Data columns (total 12 columns):
overall           int64
vote              object
verified          bool
reviewTime        object
reviewerID        object
asin              object
style             object
reviewerName      object
reviewText        object
summary           object
unixReviewTime    int64
image             object
dtypes: bool(1), int64(2), object(9)
memory usage: 572.0+ MB
None


In [2]:
df = df.dropna(subset=['reviewText'])
df.shape

(6738237, 12)

In [3]:
df_mini= df.sample(n=600)
df_mini.shape

(600, 12)

In [5]:
#df = df.dropna(subset=['style'])
#df.shape

(3546656, 12)

In [4]:
print(df_mini.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 3612555 to 1340345
Data columns (total 12 columns):
overall           600 non-null int64
vote              65 non-null object
verified          600 non-null bool
reviewTime        600 non-null object
reviewerID        600 non-null object
asin              600 non-null object
style             314 non-null object
reviewerName      600 non-null object
reviewText        600 non-null object
summary           600 non-null object
unixReviewTime    600 non-null int64
image             9 non-null object
dtypes: bool(1), int64(2), object(9)
memory usage: 56.8+ KB
None


In [5]:
# Remove ## from lines starting with ! and run them the first time to install necessary packages 

##%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)


Requirement already up-to-date: tensorflow-gpu in /home/wangjia/anaconda3/lib/python3.7/site-packages (2.1.0)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [7]:
#split the data into training and testing data, using "overall" as the target variable
y=df_mini.overall
x=df_mini.drop('overall',axis=1)


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print("x_train shape: {}".format(x_train.shape), end='\n')
print("y_train shape: {}".format(y_train.shape), end='\n\n')
print("x_test shape: {}".format(x_test.shape), end='\n')
print("y_test shape: {}".format(y_test.shape), end='\n\n')

x_train shape: (480, 11)
y_train shape: (480,)

x_test shape: (120, 11)
y_test shape: (120,)



In [11]:
reviews_messages = x_train.reviewText
message_embeddings = embed(reviews_messages)


In [12]:
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(reviews_messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

KeyError: 0

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Vectorize X_train
vectorizer = CountVectorizer(min_df=1).fit(x_train)
X_train = vectorizer.transform(x_train)
print("X_train:\n{}".format(repr(X_train)))

In [ ]:

#BoW
def apply_bow(final_df,ngram_range=(1,1)):
    count_vect = CountVectorizer(ngram_range=ngram_range) #in scikit-learn
    final_counts = count_vect.fit_transform(final_df['CleanedText'].values)
    print("the type of count vectorizer ",type(final_counts))
    print("the shape of out text BOW vectorizer ",final_counts.get_shape())
    print("the number of unique words ", final_counts.get_shape()[1])
    return final_counts  
apply_bow(df)